# Spatial Joins Exercises

Here\'s a reminder of some of the functions we have seen. Hint: they
should be useful for the exercises!

-   `sum(expression)`: aggregate to
    return a sum for a set of records
-   `count(expression)`: aggregate to
    return the size of a set of records
-   `ST_Area(geometry)` returns the
    area of the polygons
-   `ST_AsText(geometry)` returns WKT `text`
-   `ST_Contains(geometry A, geometry B)` returns the true if geometry A contains geometry B
-   `ST_Distance(geometry A, geometry B)` returns the minimum distance between geometry A and
    geometry B
-   `ST_DWithin(geometry A, geometry B, radius)` returns the true if geometry A is radius distance or less from geometry B
-   `ST_GeomFromText(text)` returns `geometry`
-   `ST_Intersects(geometry A, geometry B)` returns the true if geometry A intersects geometry B
-   `ST_Length(linestring)` returns the length of the linestring
-   `ST_Touches(geometry A, geometry B)` returns the true if the boundary of geometry A touches geometry B
-   `ST_Within(geometry A, geometry B)` returns the true if geometry A is within geometry B


Uncomment and run the following cell to install the required packages.


In [1]:
%pip install duckdb leafmap lonboard
import duckdb
import leafmap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.1/884.1 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.5/765.5 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.4/72.

Download the [nyc_data.zip](https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip) dataset using leafmap. The zip file contains the following datasets. Create a new DuckDB database and import the datasets into the database. Each dataset should be imported into a separate table.

- nyc_census_blocks
- nyc_homicides
- nyc_neighborhoods
- nyc_streets
- nyc_subway_stations

In [6]:
url = "https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip"
leafmap.download_file(url, unzip=True)

Downloading...
From: https://github.com/opengeos/data/raw/main/duckdb/nyc_data.zip
To: /content/nyc_data.zip
100%|██████████| 8.73M/8.73M [00:00<00:00, 71.1MB/s]


Extracting files...


'/content/nyc_data.zip'

In [7]:
con = duckdb.connect("nyc_data.db")

In [8]:
con.install_extension("spatial")
con.load_extension("spatial")

In [35]:
con.sql("SHOW TABLES;").df()

,name
0,nyc_census_blocks
1,nyc_homicides
2,nyc_neighborhoods
3,nyc_streets
4,nyc_subway_stations
5,simple_neighborhoods


1. **What subway station is in \'Little Italy\'? What subway route is it on?**

In [32]:
con.sql("""

SELECT *
  FROM nyc_subway_stations
  LIMIT 10

""").df()

,OBJECTID,ID,NAME,ALT_NAME,CROSS_ST,LONG_NAME,LABEL,BOROUGH,NGHBHD,ROUTES,TRANSFERS,COLOR,EXPRESS,CLOSED,geom
0,1.0,376.0,Cortlandt St,None,Church St,"Cortlandt St (R,W) Manhattan","Cortlandt St (R,W)",Manhattan,None,"R,W","R,W",YELLOW,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
1,2.0,2.0,Rector St,None,None,Rector St (1) Manhattan,Rector St (1),Manhattan,None,1,1,RED,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
2,3.0,1.0,South Ferry,None,None,South Ferry (1) Manhattan,South Ferry (1),Manhattan,None,1,1,RED,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
3,4.0,125.0,138th St,Grand Concourse,Grand Concourse,"138th St / Grand Concourse (4,5) Bronx","138th St / Grand Concourse (4,5)",Bronx,None,"4,5","4,5",GREEN,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
4,5.0,126.0,149th St,Grand Concourse,Grand Concourse,149th St / Grand Concourse (4) Bronx,149th St / Grand Concourse (4),Bronx,None,4,"2,4,5",GREEN,express,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
5,6.0,45.0,149th St,Grand Concourse,Grand Concourse,"149th St / Grand Concourse (2,5) Bronx","149th St / Grand Concourse (2,5)",Bronx,None,"2,5","2,4,5",RED-GREEN,express,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
6,7.0,127.0,161st St,Yankee Stadium,River Ave,"161st St / Yankee Stadium (B,D,4) Bronx","161st St / Yankee Stadium (B,D,4)",Bronx,None,"B,D,4","B,D,4",GREEN-ORANGE,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
7,8.0,208.0,167th St,None,Grand Concourse,"167th St (B,D) Bronx","167th St (B,D)",Bronx,None,"B,D","B,D",ORANGE,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
8,9.0,128.0,167th St,None,River Ave,167th St (4) Bronx,167th St (4),Bronx,None,4,4,GREEN,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
9,10.0,209.0,170th St,None,Grand Concourse,"170th St (B,D) Bronx","170th St (B,D)",Bronx,None,"B,D","B,D",ORANGE,None,None,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."


In [33]:
con.sql("""

SELECT * FROM nyc_neighborhoods LIMIT 10;

""").df()

,BORONAME,NAME,geom
0,Brooklyn,Bensonhurst,"[5, 4, 41, 0, 0, 0, 0, 0, 54, 71, 14, 73, 198,..."
1,Manhattan,East Village,"[5, 4, 152, 0, 0, 0, 0, 0, 35, 215, 14, 73, 13..."
2,Manhattan,West Village,"[5, 4, 91, 0, 0, 0, 0, 0, 161, 95, 14, 73, 212..."
3,The Bronx,Throggs Neck,"[5, 4, 141, 0, 0, 0, 0, 0, 128, 232, 17, 73, 1..."
4,The Bronx,Wakefield-Williamsbridge,"[5, 4, 126, 0, 0, 0, 0, 0, 83, 85, 17, 73, 17,..."
5,Queens,Auburndale,"[5, 4, 169, 0, 0, 0, 0, 0, 66, 183, 18, 73, 25..."
6,Manhattan,Battery Park,"[5, 4, 10, 0, 0, 0, 0, 0, 166, 73, 14, 73, 214..."
7,Manhattan,Carnegie Hill,"[5, 4, 120, 0, 0, 0, 0, 0, 111, 123, 15, 73, 2..."
8,Staten Island,Mariners Harbor,"[5, 4, 201, 0, 0, 0, 0, 0, 78, 45, 11, 73, 59,..."
9,Staten Island,Rossville,"[5, 4, 124, 0, 0, 0, 0, 0, 143, 219, 9, 73, 28..."


In [36]:
con.sql("""
SELECT s.name, s.routes
FROM nyc_subway_stations AS s
JOIN nyc_neighborhoods AS n
ON ST_Contains(n.geom, s.geom)
WHERE n.name = 'Little Italy';
""").df()

,NAME,ROUTES
0,Spring St,6


2. **What are all the neighborhoods served by the 6-train?** (Hint: The `routes` column in the `nyc_subway_stations` table has values like \'B,D,6,V\' and \'C,6\')


In [46]:
con.sql("""
SELECT n.name, s.routes
FROM nyc_subway_stations AS s
JOIN nyc_neighborhoods AS n
ON ST_CONTAINS(n.geom, s.geom)
WHERE s.routes LIKE '%6%';

""").df()

,NAME,ROUTES
0,Financial District,"4,5,6"
1,Chinatown,6
2,Little Italy,6
3,Greenwich Village,6
4,Greenwich Village,6
5,Greenwich Village,"4,5,6"
6,Gramercy,6
7,Gramercy,6
8,Gramercy,6
9,Murray Hill,"4,5,6"


3. **After 9/11, the \'Battery Park\' neighborhood was off limits for several days. How many people had to be evacuated?**

In [47]:
con.sql("""

SELECT * FROM nyc_homicides LIMIT 10;

""").df()

,INCIDENT_D,BORONAME,NUM_VICTIM,PRIMARY_MO,ID,WEAPON,LIGHT_DARK,YEAR,geom
0,2008-01-01,Brooklyn,1,None,7,gun,D,2008,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
1,2008-01-04,Manhattan,1,None,14,gun,D,2008,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
2,2008-01-05,Queens,1,None,15,gun,D,2008,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
3,2008-01-04,Queens,1,None,16,knife,D,2008,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
4,2008-01-05,Queens,1,None,18,gun,D,2008,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
5,2008-01-07,Brooklyn,1,None,20,gun,D,2008,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
6,2008-01-10,Manhattan,1,None,22,gun,D,2008,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
7,2008-01-10,Manhattan,1,None,23,gun,D,2008,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
8,2008-01-13,Staten Island,1,None,25,gun,D,2008,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."
9,2008-01-16,Queens,1,None,27,gun,D,2008,"[0, 0, 24, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,..."


In [48]:
con.sql("""

SELECT * FROM nyc_census_blocks LIMIT 10;

""").df()

,BLKID,POPN_TOTAL,POPN_WHITE,POPN_BLACK,POPN_NATIV,POPN_ASIAN,POPN_OTHER,BORONAME,geom
0,360850009001000,97,51,32,1,5,8,Staten Island,"[5, 4, 184, 0, 0, 0, 0, 0, 55, 3, 13, 73, 151,..."
1,360850020011000,66,52,2,0,7,5,Staten Island,"[5, 4, 136, 0, 0, 0, 0, 0, 178, 58, 13, 73, 72..."
2,360850040001000,62,14,18,2,25,3,Staten Island,"[5, 4, 120, 0, 0, 0, 0, 0, 82, 227, 12, 73, 55..."
3,360850074001000,137,92,12,0,13,20,Staten Island,"[5, 4, 184, 0, 0, 0, 0, 0, 204, 85, 13, 73, 10..."
4,360850096011000,289,230,0,0,32,27,Staten Island,"[5, 4, 89, 0, 0, 0, 0, 0, 107, 247, 12, 73, 7,..."
5,360850112011000,83,63,0,0,18,2,Staten Island,"[5, 4, 152, 0, 0, 0, 0, 0, 227, 227, 12, 73, 1..."
6,360850128041000,180,103,33,0,23,21,Staten Island,"[5, 4, 73, 0, 0, 0, 0, 0, 202, 98, 12, 73, 43,..."
7,360850138001000,22,22,0,0,0,0,Staten Island,"[5, 4, 120, 0, 0, 0, 0, 0, 169, 223, 11, 73, 1..."
8,360850146051000,48,48,0,0,0,0,Staten Island,"[5, 4, 104, 0, 0, 0, 0, 0, 35, 144, 11, 73, 11..."
9,360850146071000,0,0,0,0,0,0,Staten Island,"[5, 4, 136, 0, 0, 0, 0, 0, 110, 84, 11, 73, 56..."


In [53]:
con.sql("""
SELECT n.name, Sum(c.popn_total) AS total_evacuated
FROM nyc_census_blocks AS c
JOIN nyc_neighborhoods AS n
ON ST_INTERSECTS(n.geom, c.geom)
WHERE n.name = 'Battery Park'
GROUP BY n.name;

""").df()

,NAME,total_evacuated
0,Battery Park,17153.0


4. **What neighborhood has the highest population density (persons/km2)?**


In [69]:
con.sql("""
SELECT n.name, Sum(c.popn_total)/(ST_Area(n.geom) / 1000000.0) AS population_density
FROM nyc_census_blocks AS c
JOIN nyc_neighborhoods AS n
ON ST_INTERSECTS(n.geom, c.geom)
GROUP BY n.name, n.geom
ORDER BY population_density DESC
LIMIT 2;

""").df()

,NAME,population_density
0,North Sutton Area,68435.132838
1,East Village,50404.483413


When you're finished, you can check your answers [here](https://postgis.net/workshops/postgis-intro/joins_exercises.html).

# Ship-to-Ship Transfer Detection

Now for a less structured exercise. We're going to look at ship-to-ship transfers. The idea is that two ships meet up in the middle of the ocean, and one ship transfers cargo to the other. This is a common way to avoid sanctions, and is often used to transfer oil from sanctioned countries to other countries. We're going to look at a few different ways to detect these transfers using AIS data.

In [70]:
%pip install duckdb duckdb-engine jupysql

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 8.1 MB/s eta 0:00:00


In [71]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:

In [72]:
%%sql
INSTALL httpfs;
LOAD httpfs;
INSTALL spatial;
LOAD spatial;

,Success


## Step 1

Create a spatial database using the following AIS data:

https://storage.googleapis.com/qm2/casa0025_ships.csv

Each row in this dataset is an AIS 'ping' indicating the position of a ship at a particular date/time, alongside vessel-level characteristics.

It contains the following columns:
* `vesselid`: A unique numerical identifier for each ship, like a license plate
* `vessel_name`: The ship's name
* `vsl_descr`: The ship's type
* `dwt`: The ship's Deadweight Tonnage (how many tons it can carry)
* `v_length`: The ship's length in meters
* `draught`: How many meters deep the ship is draughting (how low it sits in the water). Effectively indicates how much cargo the ship is carrying
* `sog`: Speed over Ground (in knots)
* `date`: A timestamp for the AIS signal
* `lat`: The latitude of the AIS signal (EPSG:4326)
* `lon`: The longitude of the AIS signal (EPSG:4326)

Create a table called 'ais' where each row is a different AIS ping, with no superfluous information. Construct a geometry column.

Create a second table called 'vinfo' which contains vessel-level information with no superfluous information.

You can set a spatial index on each of these tables as follows:

`CREATE INDEX index_name ON table_name USING RTREE(geom);`

In [88]:
url = "https://storage.googleapis.com/qm2/casa0025_ships.csv"
leafmap.download_file(url)

casa0025_ships.csv already exists. Skip downloading. Set overwrite=True to overwrite.


'/content/casa0025_ships.csv'

In [74]:
import os
os.listdir('/content/')

['.config',
 'nyc_subway_stations.prj',
 'nyc_data.zip',
 'nyc_neighborhoods.shp',
 'nyc_homicides.shp',
 'nyc_data.db.wal',
 'nyc_streets.prj',
 'nyc_census_blocks.prj',
 'nyc_census_blocks.shp',
 'nyc_streets.dbf',
 'casa0025_ships.csv',
 'nyc_subway_stations.dbf',
 'nyc_homicides.dbf',
 'README.txt',
 'nyc_homicides.prj',
 'nyc_neighborhoods.dbf',
 'nyc_subway_stations.shp',
 'nyc_census_blocks.dbf',
 'nyc_streets.shx',
 'nyc_census_sociodata.sql',
 'nyc_data.db',
 'nyc_homicides.shx',
 'nyc_data.db.zip',
 'nyc_subway_stations.shx',
 'nyc_census_blocks.shx',
 'nyc_neighborhoods.shx',
 'nyc_neighborhoods.prj',
 'nyc_streets.shp',
 'sample_data']

In [85]:
con = duckdb.connect('ship.db')

In [86]:
con.install_extension("spatial")
con.load_extension("spatial")



In [87]:
con.sql("SHOW TABLES;")

┌─────────┐
│  name   │
│ varchar │
├─────────┤
│ 0 rows  │
└─────────┘

## Step 2

Use a spatial join to identify ship-to-ship transfers in this dataset.
Two ships are considered to be conducting a ship to ship transfer IF:

* They are within 500 meters of each other
* For more than two hours
* And their speed is lower than 1 knot

Some things to consider: make sure you're not joining ships with themselves. Try working with subsets of the data first while you try different things out.